In [ ]:
# Import Libraries
from nltk.corpus import stopwords
import nltk
import string
import re
import plotly.express as p
import warnings
import gc
import random
import numpy as np  
random_state = 123
from scipy.stats import pearsonr
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm_notebook
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
import matplotlib.pyplot as plt
import webbrowser
from bs4 import BeautifulSoup
import httplib2
import urllib
import requests
from pprint import pprint


In [ ]:
# Memory reduction helper function:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2
    for col in df.columns:  # columns
        col_type = df[col].dtypes
        if col_type in numerics:  # numerics
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose:
        print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(
            end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df


def varname(variable):
    for name in list(globals().keys()):
        expression = f'id({name})'
        if id(variable) == eval(expression):
            return name


Cleaning Data Function

In [ ]:
def clean_tourism_data(scraped_tourism_data):
    # lower text
    text = text.lower()
    #removing square brackets
    text = re.sub('[.*?]', '', text)
    text = re.sub('\+', '', text)
    #removing hyperlinks and emails
    text = re.sub('https?://S+|www.S+', '', text)
    text = text.rstrip('.com')
    text = text.rstrip('@gmail')
    text = re.sub(r'^https?:\/\/.*[\r\n]*', '', text, flags=re.MULTILINE)
    #removing puncuation
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    return text


Open Web Page

In [ ]:
resp = requests.get(
    'https://app.memrise.com/course/89247/kinyarwanda-phrases/1/')

soup = BeautifulSoup(resp.text, 'html.parser')

Tourism Data Scraping

In [ ]:
def scrape_tourism_data():
    kin_list = []
    eng_list = []

    for i in range(1, 27):
        resp = requests.get('https://app.memrise.com/course/89247/kinyarwanda-phrases/'+ str(i)+'/')

        soup = BeautifulSoup(resp.text, 'html.parser')

        kin_title = soup.find_all("div", attrs={"class": 'col_a col text'})
        for kin_text in kin_title:
            kin_list.append(kin_text.text)

        eng_title = soup.find_all("div", attrs={"class": 'col_b col text'})
        for eng_text in eng_title:
            eng_list.append(eng_text.text)


    ### append lists to dataframes
    kin_df =  pd.DataFrame(kin_list, columns=['kin_text'])
    eng_df = pd.DataFrame(eng_list, columns = ['eng_text'])

    return kin_df, eng_df

Health Data Scraping

In [ ]:
def scrape_health_data():
    kin_list = []
    eng_list = []

    for i in range(27, 34):
        resp = requests.get(
            'https://app.memrise.com/course/89247/kinyarwanda-phrases/' + str(i)+'/')

        soup = BeautifulSoup(resp.text, 'html.parser')

        kin_title = soup.find_all("div", attrs={"class": 'col_a col text'})
        for kin_text in kin_title:
            kin_list.append(kin_text.text)

        eng_title = soup.find_all("div", attrs={"class": 'col_b col text'})
        for eng_text in eng_title:
            eng_list.append(eng_text.text)


    ##### Another webpage on memrise
    resp = requests.get(
        'https://app.memrise.com/course/1341863/bits-of-kinyarwanda/8/')
    soup = BeautifulSoup(resp.text, 'html.parser')
    
    kin_title = soup.find_all("div", attrs={"class": 'col_a col text'})
    for kin_text in kin_title:
        kin_list.append(kin_text.text)

    eng_title = soup.find_all("div", attrs={"class": 'col_b col text'})
    for eng_text in eng_title:
        eng_list.append(eng_text.text)

    ### append lists to dataframes
    kin_df = pd.DataFrame(kin_list, columns=['kin_text'])
    eng_df = pd.DataFrame(eng_list, columns=['eng_text'])

    return kin_df, eng_df


In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
import time
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import pandas as pd


In [ ]:
#### adding options to the webdriver
chromeOptions = Options()
chromeOptions.add_argument("--kiosk")


In [6]:
driver = webdriver.Chrome(ChromeDriverManager().install())
driver.get('https://www.sportybet.com/ng/')




====== WebDriver manager ======
Current google-chrome version is 106.0.5249
Get LATEST chromedriver version for 106.0.5249 google-chrome
Driver [C:\Users\ORAEGBU AYOMIDE\.wdm\drivers\chromedriver\win32\106.0.5249.61\chromedriver.exe] found in cache


In [32]:
# Define list for saving texts

def scrape_Glosbe_Health_Data(list_of_search_words):
    kin_list =[]
    eng_list = []

    option = Options()
    option.add_argument('headless')


    # Loop through the list of search words # input the text to search in the search box
    for search_words in list_of_search_words:
        # Define chrome webDriver
        driver = webdriver.Chrome(ChromeDriverManager().install(), options=option)

        # Use the get method to load the url
        driver.get(r'https://glosbe.com/en/rw')
        driver.maximize_window()

        # Find the search box element by its name
        ele = driver.find_element_by_name('q')
        time.sleep(2)

        # Clear any text in the search box
        ele.clear()
        time.sleep(3)
        ele.send_keys(search_words)
        # Submit the form
        ele.send_keys(Keys.RETURN)

        # Use a While loop to scroll down the page
        while True:
            # find the elements which has tag name div
            Divs = driver.find_element_by_tag_name('div').text

            # if the div has does not have the text "Load more" break the loop
            if 'LOAD MORE' not in Divs:
                print("Web Scraping Complete")
                break 
                
            # otherwise find the element which has an id== "tmem_more", which is the button to load more pages using an xpath expression
            else:
                element = driver.find_element_by_xpath(
                        '//button[@id="tmem__more"]')

                # Scroll the page till you get to the element
                desired_y = (element.size['height'] / 2) + element.location['y']
                window_h = driver.execute_script('return window.innerHeight')
                window_y = driver.execute_script('return window.pageYOffset')
                current_y = (window_h / 2) + window_y
                scroll_y_by = desired_y - current_y
                driver.execute_script("window.scrollBy(0, arguments[0]);", scroll_y_by)

                # Wait for the page to load after getting to the element
                time.sleep(3)

                # Click the button to load more pages
                element = driver.find_element_by_xpath(
                        '//button[@id="tmem__more"]').click()
                    
                # wait for the page to load after clicking the button
                time.sleep(3)

                # continue the loop
                continue

        # pass the html code to BeautifulSoup and parse it using html parser    
        page_source_view = driver.page_source
        soup = BeautifulSoup(page_source_view, 'html.parser')


        ## Find all the divs with class attribute="w-1/2 relative pl-2" which is Kinyarwanda Text
        kin_text_finder = soup.find_all("div", attrs={"class": 'w-1/2 relative pl-2'})
        for kin_text in kin_text_finder:
            kin_list.append(kin_text.text.strip())



        ## Find all the divs with class attribute="w-1/2 pr-2" which is English Text
        eng_text_finder = soup.find_all("div", attrs={"class": 'w-1/2 pr-2'})
        for eng_text in eng_text_finder:
            eng_list.append(eng_text.text.strip())

    ### append lists to dataframes
    kin_df = pd.DataFrame(kin_list, columns=['kin_text'])
    eng_df = pd.DataFrame(eng_list, columns=['eng_text'])

    return kin_df, eng_df


## Scraping Health Data From Glosbe

Scraping texts that has to do with diseases, infections, sickness

In [ ]:
disease_list = ['Alzheimer', 'HIV', 'COVID', 'EBOLA', 'VIRUS', 'pneumonia',
                'flu', 'asthma', 'ulcer', 'coli', 'Gonorrhea', 'Syphilis',
                'cancer', 'leprosy', 'epilepsy', 'bacteria', 'Diarrhoea',
 'Smallpox', 'malaria', 'measles', 'chickenpox', 'yellow fever', 
 'cholera', 'fever', 'typhoid', 'tuberculosis', 'diabetes', 'heart disease',
    'lung disease', 'cancer', 'stroke', 'kidney disease', 'liver disease',
'heart attack', 'high blood pressure', 'polio', 'Meningitis',
'tuberculosis', 'herpes', 'gonorrhea', 'hepatitis','syphilis']



Scraping text that has to do with the systems of the body

In [ ]:
systems_of_body = [
 'arteries', 'veins', 'muscle', 'lungs', 'liver', 'kidney', 'plasma', 
 'platelets', 'red cells', 'white cells', 'hormones', ' pulmonary', 
 'pancreas', 'bladder', ''

]

Scraping text that has to do with Child bearing

In [ ]:
child_bearing = [
'placenta', 'MRI', ''
]

In [33]:
kin, eng = scrape_Glosbe_Health_Data(search_words)






====== WebDriver manager ======
Current google-chrome version is 103.0.5060
Get LATEST chromedriver version for 103.0.5060 google-chrome
Driver [C:\Users\ORAEGBU AYOMIDE\.wdm\drivers\chromedriver\win32\103.0.5060.134\chromedriver.exe] found in cache


Web Scraping Complete




====== WebDriver manager ======
Current google-chrome version is 103.0.5060
Get LATEST chromedriver version for 103.0.5060 google-chrome
Driver [C:\Users\ORAEGBU AYOMIDE\.wdm\drivers\chromedriver\win32\103.0.5060.134\chromedriver.exe] found in cache


Web Scraping Complete


In [15]:
(1 Peter 3:8) For instance, when a family of needy Christians had to move to a new area for health reasons, fellow believers there let them use a house rent-free for six months.

200
200


(None, None)

In [34]:
eng

,eng_text
0,"Food, water, shelter, medical care, and emotio..."
1,He told the commissioner that we had just pers...
2,Although his condition was serious and some do...
3,"For example, a woman who had spent all her res..."
4,Are You Prepared for a Medical Emergency?
...,...
395,Outline your theocratic background as well as ...
396,What a joy for them to build houses and live i...
397,6 Some health experts recommend prayer as trea...
398,A chronic health problem can make you feel as ...


In [ ]:
print(doc)